<h1>There are 2 conventions for tensor vs array elements</h1>


We write [1,2,3] with each element in an array or list separated by an comma.
<br/>
When writing a matrix we use commas to separate rows like this:

[[1 2 3],[4 5 6]].

The problem is the commas. Sometimes they separate elements in a list sometimes they separate rows in a matrix.  

In [ ]:
import torch
test = [[1,2,3 ],[4,5,6]]

x = torch.tensor(test)
m = torch.mean(x.float())
print('mean:',m.shape)
m = m.unsqueeze(0)
m = m.squeeze(1)
print("m squeeze", m.shape)

#print(m.shape)
#[3]
#[1,3]
#[3,1]

mean: torch.Size([])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
import torch
# error cell, leave error
a=torch.tensor([[1],[1],[1]]).float()
print(a.shape)


res = torch.mean(a, dim=0, keepdims=True) # if you use keepdims you need dim. dim=0 rows, dim=1 cols.
print(res.shape)

res = torch.mean(a, dim=1, keepdims=True) # if you use keepdims you need dim. dim=0 rows, dim=1 cols.
print(res.shape)

# do not fix. for demo
res = torch.mean(a, keepdims=True) # if you use keepdims you need dim. dim=0 rows, dim=1 cols.
print(res.shape)



torch.Size([3, 1])
torch.Size([1, 1])
torch.Size([3, 1])


TypeError: mean() received an invalid combination of arguments - got (Tensor, keepdims=bool), but expected one of:
 * (Tensor input, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)
 * (Tensor input, tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None, Tensor out = None)


In [ ]:
print(torch.tensor([1,2,3]).shape)
print(torch.arange(3).shape)
print(torch.ones(3).shape)
print(torch.zeros(3).shape)
print(torch.randn(3).shape)
import numpy
print(numpy.ones(3).shape)
print(numpy.zeros(3).shape)
print(numpy.random.rand(3).shape)
print(numpy.arange(3).shape)


torch.Size([3])
torch.Size([3])
torch.Size([3])
torch.Size([3])
torch.Size([3])
(3,)
(3,)
(3,)
(3,)


In [ ]:
# derive the col and row sums.

import torch
x = torch.tensor([[1,2,3], [4,5,6]])
print(f'x:{x}')


# x = (2x3)
# I=([1,1,1]x2) I@x or I=(3,[1,1]) for x@I
#
I_a = torch.tensor([1,1,1]) #
print(I_a.shape)
I_a = I_a.unsqueeze(1)
print(I_a.shape)
print(I_a)
print(x@I_a) # row or column sum?
I_b = torch.tensor([1,1])
print(f'I_b:{I_b}')
print(f'I_b shape:{I_b.shape}')
I_b = I_b.unsqueeze(0)
print(f'I_b shape:{I_b.shape}')
print(f'I_b*x:{I_b@x}')

# do matrix multiply have keepdims? No. Why? Mean have keepdims because all the elements reduce to 1 number.
#

x:tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([3])
torch.Size([3, 1])
tensor([[1],
        [1],
        [1]])
tensor([[ 6],
        [15]])
I_b:tensor([1, 1])
I_b shape:torch.Size([2])
I_b shape:torch.Size([1, 2])
I_b*x:tensor([[5, 7, 9]])


In [ ]:
#lkarpathy lets build gpt from scratch 45:00

torch.manual_seed(1337)
B,T,C = 4,8,2

x = torch.randn(B,T,C)
x.shape

#note randn creates float so we dont need conversion to float for torch.mean
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

<h1>Karpathy Prefix Average</h1>




<ul>
<li>X [1,2,3,4,5,6]</li>
<li>$\frac{1}{t+1}\sum_{t=0} x_t$</li>
<li>t=0 1/1(1)=1</li>
<li>t=1 1/2(1+2)=1.5</li>
<li>t=2 1/3(1+2+3)=2</li>
<li>t=3 1/4(1+2+3+4)=2.5</li>
</ul>

In [ ]:
# we want x[b,t] = mean_{i<t} x[b,i]
# calculate sequence average, time t=1, avg=x[1]; t=2 avg=(x[1]+x[2])/2; t=3 avg=(x[1]+x[2]+x[3])/3
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0) #calc mean on 0th dimension which is the firrst, t from (t,C) shold have keepdims=True
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

<h1>Prefix average</h1>


Prefix average where all the positions in T have the same weight. $prefixavg = \frac{1}{t+1}\sum_0^t{x_s}$

We are going to later modify the weights for each position to indicate the relative importance each position term contributes to current token.

In [ ]:
seq = torch.tensor([1,2,3,4,5,6])
for t in range(len(seq)):
    print(t,seq[:t+1])

# 0 tensor([1])                1/(0+1) sum(1)
# 1 tensor([1, 2])             1/(1+1) sum(1+2)
# 2 tensor([1, 2, 3])          1/(1+1+1) sum(1+2+3)
# 3 tensor([1, 2, 3, 4])       1/(1+1+1+1) sum(1+2+3+4)
# 4 tensor([1, 2, 3, 4, 5])    ...
# 5 tensor([1, 2, 3, 4, 5, 6])  ...

0 tensor([1])
1 tensor([1, 2])
2 tensor([1, 2, 3])
3 tensor([1, 2, 3, 4])
4 tensor([1, 2, 3, 4, 5])
5 tensor([1, 2, 3, 4, 5, 6])


<h1>Mathematical trick using Matrix Multiply and lower triangular matrix to compute prefix average</h1>

There are no weights here. Add them in next step.

In [ ]:
# karpathy mathematical trick w matric mul
# first part compute sum of rows and or columns with matrix multiply
# second part is using pytorch.tril a upper triagular matrix to act as a mask to compute prefix average
import torch

torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a@b
print('a=')
print(a)
print('-----')
print('b=')
print(b)
print('c=')
print(c) # see the column sums replicated in all rows
print('-----')



a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
-----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
-----


In [ ]:
 # to turn this into a progressive sum use triangular to mask out the future tokens

torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a@b
print('a=')
print(a)
print('-----')
print('b=')
print(b)
print('c=')
print(c) # see the column sums replicated in all rows
print('-----')

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
-----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
-----


In [ ]:

import torch.nn.functional as F

#x = torch.tensor([[1,2,3],[4,5,6],[7,8,9]]).float()
x = torch.tril(torch.ones(3,3)).float()
#print(f'x:{x}')
s = torch.sum(x, 1, keepdim=True)
print(f's:{s}')
print(s.shape)
y = x/s
print("y:",y)
c=y@b
print(c) #match karpathy 51:20
#print(F.softmax(x, dim=0)) # with ones it doesnt matter which dims


s:tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
y: tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# dont need tril do cumulative_sum
B,T,C = 1,3,3
#**tricky make sure the brackets are correct for B,T,C
x1 = torch.tensor([[
    [1,2,3],
    [4,5,6],
    [7,8,9]
]]).float()
print(f'x1:{x1},x1.shape:{x.shape}') #3,3 T,C dim_t=1, dim_c=2 we want dim=1
cum_sum = torch.cumsum(x1, dim=1)
print('cum_sum:{cum_sum}')
counts = torch.arange(1,T+1).view(1,T,1).float()
print(f'counts:{counts}')

prefix_avg = cum_sum/counts
print(prefix_avg)


# 2.5 is avg of 1+4, 4.0 is avg of 1+4+7

#tensor([[[1.0000, 2.0000, 3.0000],
#         [2.5000, 3.5000, 4.5000],
#         [4.0000, 5.0000, 6.0000]]])

x1:tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]),x1.shape:torch.Size([1, 3, 3])
cum_sum:{cum_sum}
counts:tensor([[[1.],
         [2.],
         [3.]]])
tensor([[[1.0000, 2.0000, 3.0000],
         [2.5000, 3.5000, 4.5000],
         [4.0000, 5.0000, 6.0000]]])


In [ ]:
# a vectorized  weighted average
B,T,C = 4,8,2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
print(wei)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [ ]:
xbow2 = wei @ x #(T,T) @ (B,T,C) pytorch will create batched mm by 1) adding B to dim (T,T) to (B,T,T)
# (B,T,T) @ (B,T,C) becomes (B,T,C)
# xbow2 identical to xbow

print(xbow2[0])
print(xbow[0])# verify same who proves teh vectorized version using batched matrix multiply is same as loop
# rthe prefix average uses 1s which is in tril.

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


<h6>Add weights for affinities between tokens</h6>

In [ ]:
# third version uses softmax
# karpathy the wei starts at 0
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T) #interaction between elements these are trained for now they are 0.
wei = wei.masked_fill(tril==0, float('-inf')) #tril is the mask to ignore future tokens
print(wei)
wei = F.softmax(wei, dim=1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow,xbow3)


tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [ ]:
!pip install triton

In [ ]:
import triton
import triton.language as tl

@triton.jit
def prefix_avg_kernel(
    x_ptr,        # *const float
    y_ptr,        # *float
    B, T, C,      # int32
    stride_b,     # int32 (distance in elements between consecutive batches)
    stride_t,     # int32 (distance in elements between consecutive time steps)
    stride_c,     # int32 (distance in elements between consecutive channels)
):
    # program ids: one program per (b, c)
    pid_b = tl.program_id(0)    # 0 .. B-1
    pid_c = tl.program_id(1)    # 0 .. C-1

    # guard: in case grid is bigger than B or C
    if pid_b >= B or pid_c >= C:
        return

    running_sum = tl.zeros((), dtype=tl.float32)

    # IMPORTANT:
    # Triton wants static ranges; if T is small/fixed you can make it constexpr.
    # For many practical uses you specialize the kernel on T.
    #
    # For clarity we assume T is marked constexpr in the launcher;
    # if not, you can use tl.static_range with a compile-time upper bound.
    for t in tl.static_range(0, tl.constexpr(lambda: 1024)):  # 1024 = max_T you care about
        # break if we’ve passed the real T
        if t >= T:
            break

        # Compute index into flat memory:
        # idx = pid_b * stride_b + t * stride_t + pid_c * stride_c
        idx = pid_b * stride_b + t * stride_t + pid_c * stride_c

        x_val = tl.load(x_ptr + idx)
        running_sum += x_val

        count = t + 1
        avg = running_sum / count

        tl.store(y_ptr + idx, avg)

In [ ]:
# fourth version not in karpathy

# x: (B,T,C)
B, T, C = x.shape
print(f'B:{B} T:{T} C:{C}')
out = torch.empty_like(x)

for b in range(B):
    running_sum = torch.zeros(C)
    running_count = 0
    for t in range(T):
        running_sum += x[b, t]
        running_count += 1
        out[b, t] = running_sum / running_count  # prefix average
print(f'out:{out}')

B:4 T:8 C:2
out:tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [ ]:
import triton
import triton.language as tl
import torch

@triton.jit
def prefix_avg_kernel(
    x_ptr, out_ptr,
    B, C,
    stride_b, stride_t, stride_c,
    T: tl.constexpr,   # T is compile-time constant for this kernel specialization
):
    pid = tl.program_id(0)  # 0 .. B*C-1
    b = pid // C
    c = pid % C

    if b >= B:
        return

    # base offset for this (b,c) line through time
    base_offset = b * stride_b + c * stride_c

    running_sum = tl.zeros((), dtype=tl.float32)

    # loop over time dimension, 0..T-1
    for t in tl.static_range(0, T):
        idx = base_offset + t * stride_t
        x_val = tl.load(x_ptr + idx)
        running_sum += x_val

        count = t + 1  # 1..T
        avg = running_sum / count
        tl.store(out_ptr + idx, avg)


def prefix_avg_triton(x: torch.Tensor) -> torch.Tensor:
    assert x.ndim == 3
    B, T, C = x.shape
    x = x.contiguous()
    out = torch.empty_like(x)

    stride_b, stride_t, stride_c = x.stride()

    grid = (B * C,)

    prefix_avg_kernel[grid](
        x, out,
        B, C,
        stride_b, stride_t, stride_c,
        T=T,   # passed as meta argument (constexpr)
    )
    return out

B, T, C = 1, 3, 3
x = torch.tensor([[[1., 2., 3.],
                   [4., 5., 6.],
                   [7., 8., 9.]]], device='cuda')

y = prefix_avg_triton(x)
print(y)

tensor([[[1.0000, 2.0000, 3.0000],
         [2.5000, 3.5000, 4.5000],
         [4.0000, 5.0000, 6.0000]]], device='cuda:0')


In [ ]:
# break into blocks

BLOCK_T = 128
for b in range(B):
    running_sum = torch.zeros(1, C, device=x.device)  # (1,C)
    running_count = 0

    for start in range(0, T, BLOCK_T):
        end = min(start + BLOCK_T, T)
        x_blk = x[b, start:end]                      # (Tb, C)
        # cumulative sum within block
        blk_cumsum = torch.cumsum(x_blk, dim=0)      # (Tb, C)

        # add previous running_sum
        blk_sum = blk_cumsum + running_sum           # (Tb, C)

        # counts for each position in this block
        Tb = end - start
        counts = torch.arange(
            running_count + 1,
            running_count + Tb + 1,
            device=x.device
        ).view(Tb, 1)                                # (Tb,1)

        out[b, start:end] = blk_sum / counts         # (Tb,C)

        running_sum = blk_sum[-1:]                   # final prefix sum of block
        running_count += Tb
print(f'funning_sum:{running_sum}')
print(f'running_count:{running_count}')

funning_sum:tensor([[ 8.9105, -1.3124]])
running_count:8


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [7]:
# karpathy bigram language model
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32


torch.manual_seed(1337)
with open('/content/drive/MyDrive/gpt/input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l:"".join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data) )
train_data=data[:n]
val_data = data[n:]


def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

# for b in range(batch_size):
#   for t in range(block_size):
#     context = xb[b, :t+1]
#     target = yb[b,t]
#     #print(f'when input is {context.tolist()} target:{target}')

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()

    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

    self.lm_head = nn.Linear(n_embd, vocab_size)

  # one issue is CE wants B,C,T vs B T C
  def forward(self, idx, targets=None):
    B, T ,C = idx.shape

    tok_emb = self.token_embedding_table(idx) #B T C
    logits = self.lm_head(tok_emb) # B T vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :] #B,C
      probs = F.softmax(logits, dim=-1) #B,C
      idx_next = torch.multinomial(probs, num_samples = 1) #B,1
      idx = torch.cat((idx, idx_next), dim=1) # B, T+1
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f'step:{iter} train loss:{losses['train']:.4f}, val loss:{losses['val']:.4f}')
  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))


step:0 train loss:4.3893, val loss:4.3725
step:300 train loss:2.5177, val loss:2.5493
step:600 train loss:2.4979, val loss:2.5241
step:900 train loss:2.5053, val loss:2.5147
step:1200 train loss:2.5039, val loss:2.5251
step:1500 train loss:2.4890, val loss:2.5107
step:1800 train loss:2.4895, val loss:2.5164
step:2100 train loss:2.4902, val loss:2.5157
step:2400 train loss:2.4787, val loss:2.5095
step:2700 train loss:2.4854, val loss:2.5160

A:
Wit minirush IUnonthefove thacke nd thassepe unco my wat Isavouean ye.

Cis vesis to ctha dry chi
